<a href="https://colab.research.google.com/github/AhmedMM1122/Insect-task/blob/main/Task.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [19]:
#import Libraries
import numpy as np
from sklearn.decomposition import PCA
from scipy.stats import zscore
import pandas as pd
import time
import scipy.special
from scipy.special import gamma, gammaln
from scipy.stats import mode
from scipy.stats import mode
from sklearn.preprocessing import StandardScaler,normalize
from sklearn.decomposition import PCA
import scipy.io
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Ridge
from scipy.linalg import block_diag
import argparse

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
#load dataset
dataset = '/content/drive/MyDrive/data/INSECTS' #change this donia

fname1 = f'/content/drive/MyDrive/data/INSECTS/data.mat' #change this donia
fname2 = f'/content/drive/MyDrive/data/INSECTS/splits.mat'#change this donia

data = scipy.io.loadmat(fname1)
splits = scipy.io.loadmat(fname2)

In [5]:
embeddings_dna = data['embeddings_dna']
embeddings_img = data['embeddings_img']
labels = data['labels']
trainval_loc = splits['trainval_loc']
train_loc = splits['train_loc']
test_seen_loc = splits['test_seen_loc']
test_unseen_loc = splits['test_unseen_loc']
val_seen_loc = splits['val_seen_loc']
val_unseen_loc = splits['val_unseen_loc']
G=data['G']

In [6]:
model = 'OSBC_DNA'
transductive = True

In [7]:
#Ridge_regression

def ridge_regression(embeddings_dna, embeddings_img, data_subset, lambda_,va_l1,va_l2):
    di = embeddings_img.shape[1]
    st = data_subset
    X = embeddings_img[st[va_l1], :].T
    D = embeddings_dna[st[va_l2], :].T

    DXT = np.dot(D, X.T)
    V = np.dot(DXT, np.linalg.inv(np.dot(X, X.T) + rho * np.eye(di)))

    return V

In [8]:
if transductive:
  dd = embeddings_dna.shape[1]
  di = embeddings_img.shape[1]
  rho = 1
  if model == 'OSBC_IMG':
    print("Transductive model works well if the mapping is from Image to DNA,")
    print("Thus we automatically run this version!")
  model = 'OSBC_DIT'
  st = np.concatenate((trainval_loc, test_unseen_loc, test_seen_loc), axis=1)
  embeddings_dna = zscore(embeddings_dna, axis=0)
  embeddings_img = zscore(embeddings_img, axis=0)
  st = np.array(st).flatten()
  valid_indices = (st >= 0) & (st < embeddings_img.shape[0])
  valid_indices1 = (st >= 0) & (st < embeddings_dna.shape[0])
  start_time = time.time()
  V = ridge_regression(embeddings_dna, embeddings_img, st, rho,valid_indices,valid_indices1)
  end_time = time.time()
  elapsed_time = end_time - start_time
  print('Time took for learning map in transductive setup:', elapsed_time)

Time took for learning map in transductive setup: 4.340265512466431


In [9]:
pca_dim = 500

In [10]:
def load_tuned_params(model):
 #We use the upper() method to convert the input model string to uppercase
    model = model.upper()
 #We define the parameter sets OSBC_IMG, OSBC_DNA, OSBC_DIL, and OSBC_DIT as lists.
    OSBC_IMG = [0.1, 10, 5 * pca_dim, 1]
    OSBC_DNA = [0.1, 10, 25 * pca_dim, 0.5]
    OSBC_DIL = [0.1, 10, 5 * pca_dim, 0.5]
    OSBC_DIT = [0.1, 10, 25 * pca_dim, 0.5]
#evluate model of data, if it is img,dna,.......
    if model == 'OSBC_IMG':
        data = OSBC_IMG
    elif model == 'OSBC_DNA':
        data = OSBC_DNA
    elif model == 'OSBC_DIL':
        data = OSBC_DIL
    elif model == 'OSBC_DIT':
        data = OSBC_DIT
    else:
        raise ValueError("Invalid model name")
#The selected parameters are unpacked into k_0, k_1, m, and s, and these values are returned as a tuple
    k_0, k_1, m, s = data
    return k_0, k_1, m, s


In [11]:
k_0, k_1, m, s = load_tuned_params(model)

In [12]:
tuning = False

In [13]:
def data_split(embeddings_dna, embeddings_img, labels, trainval_loc, train_loc, test_seen_loc, test_unseen_loc, val_seen_loc, val_unseen_loc, split_type, model):
  if model == 'OSBC_IMG':
    features = embeddings_img
  else:
    features = embeddings_dna

  if split_type == 'tuning':
    train_idx = train_loc
    test_seen_idx = val_seen_loc
    test_unseen_idx = val_unseen_loc
  else:
    train_idx = trainval_loc
    test_seen_idx = test_seen_loc
    test_unseen_idx = test_unseen_loc

# Training data and labels
  xtrain = features[train_idx, :]
  ytrain = labels[train_idx]

# Test data and labels, Seen and Unseen
  valid_test_seen_idx = np.where(test_seen_idx < features.shape[0])
  valid_test_seen_idx = valid_test_seen_idx[0]  # Access the first element of the tuple
  xtest_seen = features[valid_test_seen_idx, :]
  ytest_seen = labels[valid_test_seen_idx]

  xtest_unseen = features[test_unseen_idx, :]
  ytest_unseen = labels[test_unseen_idx]

  return xtrain, ytrain, xtest_unseen, ytest_unseen, xtest_seen, ytest_seen


In [14]:
x_tr, y_tr, x_ts_us, y_ts_us, x_ts_s, y_ts_s = data_split(embeddings_dna, embeddings_img, labels, trainval_loc, train_loc, test_seen_loc,
                                                          test_unseen_loc, val_seen_loc, val_unseen_loc, 'test', model);

In [15]:
x_tr = x_tr.reshape(-1, 500)  # Reshape to (19420, 500)
y_tr = y_tr.reshape(-1, 19420)  # Reshape to (19420, 1)


In [16]:
y_tr = y_tr.reshape(-1, 1)  # Reshape to (19420, 1)

In [17]:
x_ts_us = x_ts_us.reshape(-1, 500)  # Reshape to (8463, 500)
y_ts_us = y_ts_us.reshape(-1, 8463)  # Reshape to (8463, 1)

In [18]:
y_ts_us = y_ts_us.reshape(-1, 1)  # Reshape to (8463, 1)

In [22]:
#function of  calculates the prior mean and prior covarince in advance to feed in Bayesian classifier
import numpy as np
import pandas as pd
#takes Data (the data matrix) and labels (class labels) as input arguments.
def calculate_priors(Data, labels):
    _, dim = Data.shape
    unique_labels = np.unique(labels)
    num_classes = len(unique_labels)

    scatters = np.zeros((dim, dim, num_classes))
    class_means = np.zeros((num_classes, dim))

    for j in range(num_classes):
        class_data = Data[labels == unique_labels[j]]
#We compute the covariance matrices for each class using np.cov.
#The rowvar=False argument specifies that each column represents a variable
        scatters[:, :, j] = np.cov(class_data, rowvar=False)
        class_means[j, :] = np.mean(class_data, axis=0)
#we calculate the overall scatter matrix as the mean of the per-class scatter matrices and the overall class mean as the mean of the class means.
    scatter = np.mean(scatters, axis=2)
    mu_0 = np.mean(class_means, axis=0)

    return mu_0, scatter



In [23]:
Data = np.random.rand(100, 5)  # Example data with 100 samples and 5 features
labels = np.random.randint(0, 3, 100)  # Example labels (0, 1, or 2)

# Call the calculate_priors function with your data and labels
mu_0, scatter = calculate_priors(Data, labels)

# Print the calculated priors and scatter matrix
print("mu_0 (class means):")
print(mu_0)
print("\nScatter matrix:")
print(scatter)

mu_0 (class means):
[0.48654197 0.578176   0.53070621 0.50068931 0.4540872 ]

Scatter matrix:
[[ 0.07912292  0.00037229  0.00748027 -0.00042965 -0.00425852]
 [ 0.00037229  0.084484   -0.01021951  0.00390598  0.00115693]
 [ 0.00748027 -0.01021951  0.07437902 -0.00169474  0.00147164]
 [-0.00042965  0.00390598 -0.00169474  0.09331474  0.00181531]
 [-0.00425852  0.00115693  0.00147164  0.00181531  0.07345223]]
